In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import collections

# Load the dataset
data_dir = 'data'
X = np.load(f'{data_dir}/X.npy')
y = np.load(f'{data_dir}/y.npy')

# Check the distribution of labels
label_distribution = collections.Counter(y)
print(f"Label distribution: {label_distribution}")

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")